In [13]:
import cv2
import numpy
from ultralytics import YOLO
import easyocr
import re
from collections import defaultdict, deque

# load models

In [14]:
model = YOLO(r"D:\Belajar FR\Youtube_2\plate-detection\model_plate_detection.pt")
reader = easyocr.Reader(['en'])

# dibawah ini pattern plat negara (regex = reguler expression)
plate_pattern = re.compile(r'^[A-Z]{2}[0-9]{2}[A-Z]{3}$') # 2 huruf, 2 angka, 3 huruf 
# kalau ingin memberikan spasi dengan /s? (boleh 0 atau 1 contoh bisa membaca 12A 123A atau 12 A 123 A)


# mapping untuk angka dan huruf terdekat

In [15]:
def correct_plate_format(ocr_text):
    mapping_num_to_alpha = {"0":"O","1":"I","5":"S","8":"B"}
    mapping_alpha_to_num = {"O":"0","I":"1","Z":"2","S":"5","B":"8"}

    ocr_text = ocr_text.upper().replace(" ","")
    if len(ocr_text) != 7:
        return "" #discard if wrong length
    
    corrected = []
    for i, ch in enumerate (ocr_text):
        if i<2 or i >= 4: # posisi alfabet (jika 0,1 adalah alfabet, jika 4,5 adalah alfabet dan 2,3 adalah angka) contoh: 12A123A
            if ch.isdigit() and ch in mapping_num_to_alpha:
                corrected.append(mapping_num_to_alpha[ch])
            elif ch.isalpha():
                corrected.append(ch)
            else:
                return "" # invalid char
            
        else: # posisi numerik
            if ch.isalpha() and ch in mapping_alpha_to_num:
                corrected.append(mapping_alpha_to_num[ch])
            elif ch.isdigit():
                corrected.append(ch)
            else:
                return "" # invalid char
    return "".join(corrected)


simulasi = "b8 12s ab"
print(correct_plate_format(simulasi))

BB12SAB


# preprocessing plat nomor sebelum ocr

In [16]:
def recognize_plate(plate_crop):
    if plate_crop.size == 0:
        return ""
    
    # preproses untuk OCR
    gray = cv2.cvtColor(plate_crop, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    plate_resized = cv2.resize(thresh, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

    try:
        ocr_result = reader.readtext(
            plate_resized,
            allowlist="0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ",
            detail=0
        )
        if len(ocr_result) > 0:
            candidate = correct_plate_format(ocr_result[0])
            if candidate and plate_pattern.match(candidate):
                return candidate
    except:
        pass

    return ""

# stabilisasi buffer untuk plat nomornya

In [17]:
plate_history = defaultdict(lambda: deque(maxlen=10)) 
# prediksi per box
plate_final = {}

def get_box_id(x1, y1, x2, y2):
    return (f"{int(x1)}_{int(y1)}_{int(x2)}_{int(y2)}") # contoh: "x1, y1, x2, y2")
def get_stable_plate(box_id, new_text):
    if new_text:
        plate_history[box_id].append(new_text)

        #voting untuk mayoritas
        most_common = max(set(plate_history[box_id]), key=plate_history[box_id].count)
        plate_final[box_id] = most_common
    return plate_final.get(box_id, "")                          


# inference video

In [18]:
input_video = r"D:\Belajar FR\Youtube_2\plate-detection\sample.mp4"
output_video = "output.mp4"

cap = cv2.VideoCapture(input_video)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, 
                      cap.get(cv2.CAP_PROP_FPS), 
                      (int(cap.get(3)), int(cap.get(4))))

CONF_THRESH = 0.3

# operating frame by frame

In [19]:
while cap.isOpened():
    succes, frame = cap.read()
    if not succes:
        break

    results=model(frame, verbose = False)

    for r in results:
        boxes = r.boxes
        for box in boxes:
            conf = float(box.conf.cpu().numpy())
            if conf < CONF_THRESH:
                continue

            x1,y1,x2,y2 = map(int, box.xyxy.cpu().numpy()[0])
            plate_crop = frame[y1:y2, x1:x2]

            #koreksi dengan ocr
            text = recognize_plate(plate_crop)

            #stabilisasi text dengan history
            box_id = get_box_id(x1, y1, x2, y2)
            stable_text = get_stable_plate(box_id, text)

            cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 3)

            if plate_crop.size > 0:
                overlay_h, overlay_w = 150, 400
                plate_resized = cv2.resize(plate_crop, (overlay_w, overlay_h))

                oy1 = max(0, y1 - overlay_h - 400)
                ox1 = x1
                oy2, ox2 = oy1 + overlay_h, ox1 + overlay_w

                if oy2 <= frame.shape[0] and ox2 <= frame.shape[1]:
                    frame[oy1:oy2, ox1:ox2] = plate_resized

                    if stable_text:
                        cv2.putText(
                            frame,
                            stable_text,
                            (ox1, oy1 - 20),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            2,
                            (0, 0, 0),
                            6
                        )
                        cv2.putText(
                            frame,
                            stable_text,
                            (ox1, oy1 - 20),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            2,
                            (255, 255, 255),
                            3
                        )

    out.write(frame)
    cv2.imshow("Plate Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
out.release()
cap.destroyAllWindows()


C:\Users\EIZY\AppData\Local\Temp\ipykernel_6448\255656283.py:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  conf = float(box.conf.cpu().numpy())


error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
